In [1]:
require 'bio'

true

In [66]:
# Retrieve sequences.
fasta_file = Bio::FlatFile.auto("./data/TAIR10_cds_20101214_updated.fa")
fasta_file.each_entry do |x|
    p x.naseq.translate[1..-2]
    abort
    protein = x.naseq.translate
    puts protein.to_fasta(header= x.entry_id)
    abort
end

"VGGKKKTKICDKVSHEEDRISQLPEPLISEILFHLSTKDSVRTSALSTKWRYLWQSVPGLDLDPYASSNTNTIVSFVESFFDSHRDSWIRKLRLDLGYHHDKYDLMSWIDAATTRRIQHLDVHCFHDNKIPLSIYTCTTLVHLRLRWAVLTNPEFVSLPCLKIMHFENVSYPNETTLQKLISGSPVLEELILFSTMYPKGNVLQLRSDTLKRLDINEFIDVVIYAPLLQCLRAKMYSTKNFQIISSGFPAKLDIDFVNTGGRYQKKKVIEDILIDISRVRDLVISSNTWKEFFLYSKSRPLLQFRYISHLNARFYISDLEMLPTLLESCPKLESLILVMSSFNPS"


In [89]:
# Modify databases
fastaoutput = File.open('./data/ara.fa', 'w')
fasta_file = Bio::FlatFile.auto("./data/TAIR10_cds_20101214_updated.fa")
fasta_file.each_entry do |x|
    x.definition = x.entry_id
    fastaoutput.write(x.to_s)
end

In [ ]:
# Modify databases
fastaoutput = File.open('./data/ara.fa', 'w')
fasta_file = Bio::FlatFile.auto("./data/TAIR10_cds_20101214_updated.fa")
fasta_file.each_entry do |x|å
    x.definition = x.entry_id
    fastaoutput.write(x.to_s)
end